Connected to che_env (Python 3.8.8)

# Experiment train analysis

Un notebook hecho para poder analizar resultados de una busqueda de hyperparametros con Ray

## Funciones auxiliares ploteo entrenamientos y extracción de resultados

In [2]:
import glob
import json
import os
import numpy as np
import pandas as pd
import plotly.express as px
from scipy import signal
from ray.tune import ExperimentAnalysis


def training_data_experiment(experiment_dir:str,trials_id:list,metrics:list,windows_size:int,width=850,height=800):

    data_to_plot = pd.DataFrame(columns=["Hyperparameters","trial_id","checkpoints","epoch"]+metrics)

    for trial_id in trials_id:
        progress_file_id_list = glob.glob(os.path.join(experiment_dir,f"TorchTrainer_{trial_id}*","progress.csv"))
        progress_file_id = progress_file_id_list[0]
        progress_df = pd.read_csv(progress_file_id)

        checkpoints_str = [checkpoint.split("/")[-1] for checkpoint in glob.glob(os.path.join(experiment_dir,f"TorchTrainer_{trial_id}*","checkpoint*"))]
        checkpoints_int = [int(checkpoint.split("_")[1]) for checkpoint in checkpoints_str]

        hparams_json_paths = glob.glob(os.path.join(experiment_dir,f"TorchTrainer_{trial_id}*","params.json"))
        with open(hparams_json_paths[0]) as hpams_json:
            hpams = json.load(hpams_json)["train_loop_config"]
        
        hpams_str = f'FL1:{hpams["fully_layer_1"]} FL2:{hpams["fully_layer_2"]} DR:{hpams["drop_rate"]} BS:{hpams["batch_size"]} LR:{hpams["learning_rate"]}'

        data_from_trial = progress_df[["epoch"]+metrics]

        for metric in metrics:
            data_from_trial.loc[:,metric] = smooth(data_from_trial[metric].to_numpy(),windows_size)

        data_from_trial.loc[:,["Hyperparameters"]] = [hpams_str]*len(data_from_trial)
        data_from_trial.loc[:,["trial_id"]] = [trial_id]*len(data_from_trial)
        data_from_trial.loc[:,["checkpoints"]] = [str(checkpoints_int)]*len(data_from_trial)

        data_to_plot = pd.concat([data_to_plot,data_from_trial],axis=0)
    
    return data_to_plot

def smooth(data, window):
    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

def plot_training_curve_single_model(data_to_plot,width=850,height=800):
    '''
    Solo usar con trials multiples del mismo modelo.
    '''
     
    metrics = [metric for metric in data_to_plot.columns if metric not in ["Hyperparameters","checkpoints","trial_id","epoch"]]

    if len(metrics) > 1:
        fig = px.line(data_to_plot,x="epoch",y=metrics, line_dash = "trial_id", width=width, height=height, hover_data=["Hyperparameters","checkpoints"])
    else:
        fig = px.line(data_to_plot,x="epoch",y=metrics, color = "trial_id", width=width, height=height, hover_data=["Hyperparameters","checkpoints"])

    return fig    

def plot_training_curve_multiple_model(data_to_plot:dict,width=850,height=800):
    '''
    Solo usar con trials simples. Muchos modelos distintos, pero solo 1 trial en cada 1
    '''

    data_to_plot_compiled = pd.DataFrame()

    for key, df in data_to_plot.items():
        df.loc[:,["model"]] = key
        data_to_plot_compiled = pd.concat([data_to_plot_compiled,df],axis=0)

    metrics = [metric for metric in data_to_plot_compiled.columns if metric not in ["Hyperparameters","checkpoints","trial_id","epoch"]]
    if len(metrics) > 1:
        fig = px.line(data_to_plot_compiled,x="epoch",y=metrics, line_dash = "model", width=width, height=height, hover_data=["Hyperparameters","checkpoints"])
    else:
        fig = px.line(data_to_plot_compiled,x="epoch",y=metrics, color = "model", width=width, height=height, hover_data=["Hyperparameters","checkpoints"])

    return fig
        
def get_result_df(experiment_path):
    experiment = ExperimentAnalysis(experiment_checkpoint_path=experiment_path)
    return experiment.results_df

/big/lab/sjinich/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-06 11:15:30,773	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-08-06 11:15:30,957	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Extracción del df de resultados

In [3]:
experiment_path = "/home/sjinich/disco/TrypanoDEEPscreen/.experiments/chembl5567_auroc_max"
result_df = get_result_df(experiment_path)

## Ordenar analizar los resultados
Aca se va a elegir que trials plotear. Por ejemplo, lo que tengan el mejor MCC o el que tenga, las distintas metricas, etc.

In [9]:
# Aca me quedo con los 5 mejores trials en base al auroc de validacion
best_5_id_auroc = result_df.sort_values("val_auroc",ascending=False).head(3).index
top_5_results = training_data_experiment(experiment_path,best_5_id_auroc,["val_loss","train_loss","val_mcc","val_auroc"],windows_size=8)
fig = plot_training_curve_single_model(top_5_results)
fig

In [5]:
# Aca me quedo con los 5 mejores trials en base al auroc de validacion
best_5_id_mcc = result_df.sort_values("val_mcc",ascending=False).head(5).index
top_5_results = training_data_experiment(experiment_path,best_5_id_mcc,["val_loss","train_loss","val_mcc","val_auroc"],windows_size=8)
fig = plot_training_curve_single_model(top_5_results)
fig

## Ahora veamos la curva ROC

In [6]:
import sys
sys.path.insert(0, '/home/sjinich/disco/TrypanoDEEPscreen/src/ml')
from engine.system import DEEPScreenClassifier
from datasets.datamodule import DEEPscreenDataModule

from lightning import Trainer
import pandas as pd
import numpy as np
import glob
import os

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL262.csv")

path = "/home/sjinich/disco/TrypanoDEEPscreen/.experiments/chembl262_poster"

trial_id = best_id.item()
numero_checkpoint = 199
checkpoint_path = glob.glob(os.path.join(path,f"*{trial_id}*/checkpoint*{numero_checkpoint}/*.ckpt"))[0]

experiment_path = "/home/sjinich/disco/TrypanoDEEPscreen/.experiments"

trainer = Trainer()
model = DEEPScreenClassifier.load_from_checkpoint(checkpoint_path,experiment_result_path=experiment_path)
datamodule = DEEPscreenDataModule(data=data,
                                      batch_size=model.hparams["batch_size"],
                                      experiment_result_path=experiment_path,
                                      data_split_mode="non_random_split",
                                      tmp_imgs=False)
    
trainer.test(model,datamodule=datamodule)

NameError: name 'best_id' is not defined

In [ ]:
import plotly.express as px
from sklearn.metrics import roc_curve, auc

def roc(true,pred):
    fpr, tpr, thresholds = roc_curve(true, pred)

    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.update_yaxes(scaleanchor="x", scaleratio=1)
    fig.update_xaxes(constrain='domain')
    return fig

In [ ]:
test_normal = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.experiments/test_chembl5567_auroc_max_256-256-0.0001-0.3-32.csv")
roc(test_normal["label"],test_normal["1_active_probability"])